In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# function courtesy of CryptoDataDownload.com
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
                            
        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'cb_{pair_split[0] + pair_split[1]}_daily.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

<strong>Pulling Bitcoin</strong><br>
To get a baseline, first working with Bitcoin.

In [3]:
fetch_daily_data('BTC/USD')

In [4]:
btc = pd.read_csv('cb_BTCUSD_daily.csv', index_col='date', parse_dates=[0])

In [5]:
btc.head()

,unix,low,high,open,close,volume,vol_fiat
date,,,,,,,
2021-01-13,1610496000,32309.04,35919.30,34035.53,35700.00,28688.617198,1.024184e+09
2021-01-12,1610409600,32500.00,36604.51,35456.89,34038.98,48227.627520,1.641619e+09
2021-01-11,1610323200,30100.00,38273.88,38168.89,35452.59,102503.156728,3.634002e+09
2021-01-10,1610236800,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,1610150400,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09


In [6]:
btc.drop('unix', axis=1, inplace=True)

In [7]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 2021-01-13 to 2020-03-20
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   low       300 non-null    float64
 1   high      300 non-null    float64
 2   open      300 non-null    float64
 3   close     300 non-null    float64
 4   volume    300 non-null    float64
 5   vol_fiat  300 non-null    float64
dtypes: float64(6)
memory usage: 16.4+ KB


In [8]:
btc.index = pd.to_datetime(btc.index)

In [9]:
btc.head()

,low,high,open,close,volume,vol_fiat
date,,,,,,
2021-01-13,32309.04,35919.30,34035.53,35700.00,28688.617198,1.024184e+09
2021-01-12,32500.00,36604.51,35456.89,34038.98,48227.627520,1.641619e+09
2021-01-11,30100.00,38273.88,38168.89,35452.59,102503.156728,3.634002e+09
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09


In [10]:
btc.head()

,low,high,open,close,volume,vol_fiat
date,,,,,,
2021-01-13,32309.04,35919.30,34035.53,35700.00,28688.617198,1.024184e+09
2021-01-12,32500.00,36604.51,35456.89,34038.98,48227.627520,1.641619e+09
2021-01-11,30100.00,38273.88,38168.89,35452.59,102503.156728,3.634002e+09
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09


In [11]:
btc = btc.sort_values('date')

In [12]:
def make_features(data):
    data['range'] = data['high'] - data['low']
    data['%change'] = (1 - (data['high'] / data['low'])) * -1
    data['month'] = data.index.month
    data['dayofweek'] = data.index.dayofweek
    data['week_low_mean'] = data['low'].shift().rolling(7).mean()
    data['week_high_mean'] = data['high'].shift().rolling(7).mean()
    data['month_low_mean'] = data['low'].shift().rolling(30).mean()
    data['month_high_mean'] = data['high'].shift().rolling(30).mean()

In [13]:
make_features(btc)

In [15]:
btc.tail(14)

,low,high,open,close,volume,vol_fiat,range,%change,month,dayofweek,week_low_mean,week_high_mean,month_low_mean,month_high_mean
date,,,,,,,,,,,,,,
2020-12-31,28000.00,29321.90,28897.42,28990.08,28813.886911,8.353169e+08,1321.90,0.047211,12,3,25089.014286,26808.140000,20918.319000,22180.004000
2021-01-01,28700.00,29688.88,28990.08,29412.84,22211.252518,6.532960e+08,988.88,0.034456,1,4,25844.315714,27599.125714,21248.019000,22493.562667
2021-01-02,29039.00,33300.00,29413.29,32225.91,46675.246521,1.504152e+09,4261.00,0.146734,1,5,26599.172857,28301.687143,21593.519000,22838.525333
2021-01-03,32008.62,34810.00,32222.88,33080.66,36951.716506,1.222387e+09,2801.38,0.087520,1,6,27249.028571,29227.115714,21931.985667,23294.337333
2021-01-04,27678.00,33666.99,33082.84,32019.99,46045.389685,1.474373e+09,5988.99,0.216381,1,0,28139.895714,30144.687143,22379.738000,23803.122000
2021-01-05,29891.13,34499.67,32020.22,34030.64,42282.569200,1.438903e+09,4608.54,0.154178,1,1,28367.297143,31028.971429,22685.638000,24285.689000
2021-01-06,33352.54,37000.00,34043.91,36859.26,45744.103200,1.686094e+09,3647.46,0.109361,1,2,28946.925714,32043.671429,23052.742333,24788.003000
2021-01-07,36200.00,40425.00,36859.26,39505.56,50346.305691,1.988959e+09,4225.00,0.116713,1,3,29809.898571,33183.920000,23534.460333,25373.584000
2021-01-08,36565.08,41986.37,39510.55,40665.15,48522.484903,1.973174e+09,5421.29,0.148264,1,4,30981.327143,34770.077143,24134.460333,26077.767000


# Expirimenting with Bitcoin

Before trying to create any models, the first test is to play with the data to see if it's theoretically possible to generate profits trading lows and highs. 

I will start by focusing on just the last month of data.

In [21]:
btc_test = btc[268:]

In [22]:
btc_test

,low,high,open,close,volume,vol_fiat,range,%change,month,dayofweek,week_low_mean,week_high_mean,month_low_mean,month_high_mean
date,,,,,,,,,,,,,,
2020-12-13,18730.00,19421.03,18821.28,19166.65,10624.200692,2.036303e+08,691.03,0.036894,12,6,18160.938571,18944.928571,17584.155667,18585.957333
2020-12-14,18967.68,19349.98,19166.65,19272.37,9102.369726,1.754242e+08,382.30,0.020155,12,0,18139.795714,18943.611429,17675.989000,18683.594333
2020-12-15,19051.27,19565.00,19272.37,19444.60,15345.212127,2.983815e+08,513.73,0.026966,12,1,18149.321429,18931.812857,17784.637000,18783.940333
2020-12-16,19300.00,21569.94,19443.19,21359.65,33350.235002,7.123493e+08,2269.94,0.117613,12,2,18270.931429,18969.740000,17893.143000,18896.920333
2020-12-17,21252.01,23776.94,21360.50,22826.48,53602.349524,1.223553e+09,2524.93,0.118809,12,3,18508.217143,19387.255714,18007.176333,19052.851667
2020-12-18,22329.00,23280.00,22826.37,23137.76,24829.235818,5.744929e+08,951.00,0.042590,12,4,18986.361429,20133.044286,18163.062667,19249.416333
2020-12-19,22770.00,24200.00,23138.89,23849.99,20577.664412,4.907771e+08,1430.00,0.062802,12,5,19664.790000,20845.461429,18333.862000,19409.149667
2020-12-20,23100.00,24300.00,23850.00,23476.51,14963.417871,3.512888e+08,1200.00,0.051948,12,6,20342.851429,21594.698571,18514.328667,19609.373333
2020-12-21,21913.84,24118.75,23476.51,22729.40,25728.638560,5.847965e+08,2204.91,0.100617,12,0,20967.137143,22291.694286,18692.170000,19791.696667


In [34]:
btc_test.drop(['open', 'close', 'volume', 'vol_fiat', 'range', 'month', 'dayofweek', 'week_low_mean', 
                'week_high_mean', 'month_low_mean', 'month_high_mean'], axis=1, inplace=True)

C:\Users\raide\OneDrive\Documents\python_projects\env\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [35]:
btc_test

,low,high,%change
date,,,
2020-12-13,18730.00,19421.03,0.036894
2020-12-14,18967.68,19349.98,0.020155
2020-12-15,19051.27,19565.00,0.026966
2020-12-16,19300.00,21569.94,0.117613
2020-12-17,21252.01,23776.94,0.118809
2020-12-18,22329.00,23280.00,0.042590
2020-12-19,22770.00,24200.00,0.062802
2020-12-20,23100.00,24300.00,0.051948
2020-12-21,21913.84,24118.75,0.100617


In [36]:
def buy_sell(data, amount):
    '''Data: Takes a dataframe with date, high, low, and %change columns. 
    Amount: Takes a numerical value (default is 1000), which represents the money being invested.
    Returns a dataframe with appended columns indicating amount invested and amount returned'''
    data['investment'] = amount
    data['profit_loss'] = 0 #setting dummy variable to save work before proceeding
    